In [29]:
import pandas as pd  # type: ignore
import matplotlib.pyplot as plt  # type: ignore
import numpy as np  # type: ignore

In [30]:
df = pd.read_csv('./../processed_data/filtered_data_with_classes.csv')
print(df.head(1))

  day_id city_id  temperature_celsius  condition_text  wind_kph  wind_degree  \
0  D0003    C001              0.46875               0       0.0     0.016854   

   pressure_mb  precip_mm  humidity  cloud  feels_like_celsius  visibility_km  \
0     0.470588   0.229592       1.0   0.75            0.371257       0.308176   

   uv_index  gust_kph  air_quality_us-epa-index  sunrise_num  sunset_num  
0       0.0  0.187697                       0.0     0.302469    0.527638  


In [31]:
from sklearn.preprocessing import LabelEncoder  # type: ignore
from sklearn.model_selection import train_test_split  # type: ignore
from sklearn.metrics import accuracy_score  # type: ignore
from xgboost import XGBClassifier  # type: ignore
from sklearn.model_selection import train_test_split

In [32]:
# Encoding categorical variables (if 'condition_text' is categorical)
label_encoder = LabelEncoder()
df['condition_text'] = label_encoder.fit_transform(df['condition_text'])
df['city_id'] = label_encoder.fit_transform(df['city_id'])

# Separating features and target variable
X = df.drop(['condition_text', "day_id"], axis=1)  # Features
y = df['condition_text']  # Target

# Step 2: Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

In [40]:
model = XGBClassifier(eval_metric='mlogloss')
print(model.get_xgb_params())
model.fit(X_train, y_train)

{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'eval_metric': 'mlogloss', 'gamma': None, 'grow_policy': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'monotone_constraints': None, 'multi_strategy': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [42]:
from sklearn.model_selection import GridSearchCV

model = XGBClassifier(eval_metric='mlogloss')

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],  # Fraction of features used per tree
    # Minimum sum of instance weight (hessian) needed in a child
    'min_child_weight': [1, 2, 3],
    # Minimum loss reduction required to make a further partition on a leaf node
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.01, 0.1],  # L1 regularization term on weights
    'reg_lambda': [1, 0.01, 0.1]  # L2 regularization term on weights
}

# Instantiate the GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy found: ", grid_search.best_score_)

Fitting 3 folds for each of 243 candidates, totalling 729 fits


ValueError: 
All the 729 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Panamgama\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Panamgama\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\Panamgama\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\sklearn.py", line 1531, in fit
    self._Booster = train(
                    ^^^^^^
  File "c:\Users\Panamgama\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgbo

In [34]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7083


In [35]:
df_input = pd.read_csv('./../processed_data/filtered_input.csv')
print(df_input.head(1))

  day_id city_id  temperature_celsius  wind_kph  wind_degree  pressure_mb  \
0  D0001    C001             0.617021  0.035411     0.582173     0.444444   

   precip_mm  humidity  cloud  feels_like_celsius  visibility_km  uv_index  \
0        0.0  0.510638   0.75            0.414634         0.3125  0.555556   

   gust_kph  air_quality_us-epa-index  sunrise_num  sunset_num  
0  0.149533                       0.2     0.311765    0.542714  


In [36]:
df_input_processed = df_input.drop(['day_id'], axis=1)
df_input_processed['city_id'] = label_encoder.fit_transform(df_input_processed['city_id'])

# Make predictions (assuming model is already trained and available)
# predictions = model.predict(df_input_processed)

In [37]:
# probabilities = model.predict_proba(df_input_processed)
# predicted_classes = np.argmax(probabilities, axis=1)

In [38]:
lower_bound = 0.85
upper_bound = 0.95
step = 0.02

# Generate confidence levels
confidence_levels = np.arange(lower_bound, upper_bound + step, step).tolist()
# confidence_levels = [95]

for confidence_threshold in confidence_levels:
    probabilities = model.predict_proba(df_input_processed)
    # print(probabilities.shape)
    # print(probabilities[:5])
    predicted_classes = np.argmax(probabilities, axis=1)
    # print(predicted_classes.shape)
    # print(predicted_classes[:5])
    # Filter records based on the current confidence threshold
    max_probabilities = np.max(probabilities, axis=1)
    # print(max_probabilities.shape)
    sure_indices = np.where(max_probabilities >= confidence_threshold)[0]
    print("sure count", sure_indices.shape)
    sure_records_df = df_input_processed.iloc[sure_indices].copy()
    sure_labels = predicted_classes[sure_indices]
    sure_records_df['predicted_labels'] = sure_labels

    # Split the new dataset into training and testing subsets
    X_sure = sure_records_df.drop('predicted_labels', axis=1)
    y_sure = sure_records_df['predicted_labels']

    # Merge with previous training data
    X_added = pd.concat([X_train, X_sure], ignore_index=True)
    y_added = pd.concat([y_train, y_sure], ignore_index=True)

    # Retrain the model on this new training subset
    model_retrained = XGBClassifier(eval_metric='logloss')
    model_retrained.fit(X_added, y_added)
    y_pred = model_retrained.predict(X_test)

    # Evaluate and print accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Confidence Threshold: {confidence_threshold: .2f}, Accuracy: {accuracy:.4f}")

sure count (1889,)
Confidence Threshold:  0.85, Accuracy: 0.6667
sure count (1856,)
Confidence Threshold:  0.87, Accuracy: 0.6667
sure count (1799,)
Confidence Threshold:  0.89, Accuracy: 0.6875
sure count (1751,)
Confidence Threshold:  0.91, Accuracy: 0.6875
sure count (1675,)
Confidence Threshold:  0.93, Accuracy: 0.6667
sure count (1575,)
Confidence Threshold:  0.95, Accuracy: 0.6875


In [459]:
# Step 2 & 3: Filter records based on confidence threshold and append predicted labels
confidence_threshold = 0.95
max_probabilities = np.max(probabilities, axis=1)
sure_indices = np.where(max_probabilities >= confidence_threshold)[0]
sure_records_df = df_input_processed.iloc[sure_indices].copy()
sure_labels = predicted_classes[sure_indices]

# Create a new DataFrame with sure records and their predicted labels
sure_records_df['predicted_labels'] = sure_labels

# Step 4: Split this new dataset into training and testing subsets
X_sure = sure_records_df.drop('predicted_labels', axis=1)
y_sure = sure_records_df['predicted_labels']

X_added = pd.concat([X_train, X_sure], ignore_index=True)
y_added = pd.concat([y_train, y_sure], ignore_index=True)

# Step 5: Retrain the model on this new training subset
model_retrained = XGBClassifier(eval_metric='logloss')
model_retrained.fit(X_added, y_added)
y_pred = model_retrained.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [460]:
y_pred = model_retrained.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7292


In [432]:
print(sure_records_df.count())
print(df_input_processed.count())

city_id                     1653
temperature_celsius         1653
wind_kph                    1653
wind_degree                 1653
pressure_mb                 1653
precip_mm                   1653
humidity                    1653
cloud                       1653
feels_like_celsius          1653
visibility_km               1653
uv_index                    1653
gust_kph                    1653
air_quality_us-epa-index    1653
sunrise_num                 1653
sunset_num                  1653
predicted_labels            1653
dtype: int64
city_id                     2414
temperature_celsius         2414
wind_kph                    2414
wind_degree                 2414
pressure_mb                 2414
precip_mm                   2414
humidity                    2414
cloud                       2414
feels_like_celsius          2414
visibility_km               2414
uv_index                    2414
gust_kph                    2414
air_quality_us-epa-index    2414
sunrise_num                 24

In [433]:
import json
with open('./../unique_dict.json', 'r') as file:
    prediction_encoder = json.load(file)

print(prediction_encoder)
prediction_decoder = {v: k for k, v in prediction_encoder.items()}
print(prediction_decoder)

{'Light Rain with Thunder': 0, 'Clear and Sunny': 1, 'Partly Cloudy': 2, 'Light Precipitation': 3, 'Cloudy and Overcast': 4, 'Mist or Fog': 5, 'Rain Showers': 6, 'Moderate to Heavy Rain': 7, 'Thunderstorms': 8}
{0: 'Light Rain with Thunder', 1: 'Clear and Sunny', 2: 'Partly Cloudy', 3: 'Light Precipitation', 4: 'Cloudy and Overcast', 5: 'Mist or Fog', 6: 'Rain Showers', 7: 'Moderate to Heavy Rain', 8: 'Thunderstorms'}


In [434]:
predictions[:10]

array([2, 2, 1, 1, 2, 2, 2, 1, 1, 1], dtype=int64)

In [435]:
# Decode predictions (assuming you have a dict named 'prediction_decoder' for this purpose)
decoded_predictions = [prediction_decoder[pred] for pred in predictions]

# Add decoded predictions back to df_input
df_input['predictions'] = decoded_predictions
daily_data = pd.read_csv('./../data/daily_data.csv')
daily_data['condition_flag'] = daily_data['condition_text'].isna().astype(int)

temp_df = df_input[['city_id', 'day_id', 'predictions']].copy()

result = pd.merge(daily_data, temp_df, on=['city_id', 'day_id'], how='left')

result['condition_text'] = result['condition_text'].fillna(result['predictions'])

result.drop('predictions', axis=1, inplace=True)

In [436]:
result.to_csv('./../output/predictions_with_data.csv', index=False)

In [437]:
result.to_csv('./../output/predictions_with_data.csv', index=False)
result[["day_id", "condition_text"]].to_csv('./../output/submit.csv', index=False)


In [438]:
print(result["condition_text"].value_counts())

condition_text
Clear and Sunny            1206
Partly Cloudy              1158
Light Precipitation         179
Mist or Fog                 138
Cloudy and Overcast          99
Rain Showers                 34
Thunderstorms                33
Light Rain with Thunder      23
Moderate to Heavy Rain       23
Name: count, dtype: int64
